In [1769]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F

In [1770]:
data = pd.read_csv('pytorched/data/processed_data.csv')

In [1771]:
viewport = data.copy()
viewport[(viewport.season == 2021) & (viewport['round'] == 3)][:22]

,season,round,country,url,driver,grid,podium,driver_points,driver_wins,driver_standings_pos,...,constructor_haas,constructor_lotus_f1,constructor_mclaren,constructor_mercedes,constructor_red_bull,constructor_renault,constructor_sauber,constructor_toro_rosso,constructor_toyota,constructor_williams
7059,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,bottas,1,3,32.0,0.0,4.0,...,False,False,False,True,False,False,False,False,False,False
7060,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,hamilton,2,1,69.0,2.0,1.0,...,False,False,False,True,False,False,False,False,False,False
7061,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,max_verstappen,3,2,61.0,1.0,2.0,...,False,False,False,False,True,False,False,False,False,False
7062,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,perez,4,4,22.0,0.0,6.0,...,False,False,False,False,True,False,False,False,False,False
7063,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,sainz,5,11,14.0,0.0,8.0,...,False,False,False,False,False,False,False,False,False,False
7064,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,ocon,6,7,8.0,0.0,9.0,...,False,False,False,False,False,False,False,False,False,False
7065,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,norris,7,5,37.0,0.0,3.0,...,False,False,True,False,False,False,False,False,False,False
7066,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,leclerc,8,6,28.0,0.0,5.0,...,False,False,False,False,False,False,False,False,False,False
7067,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,gasly,9,10,7.0,0.0,10.0,...,False,False,False,False,False,False,False,False,False,False
7068,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,vettel,10,13,0.0,0.0,16.0,...,False,False,False,False,False,False,False,False,False,False


In [1772]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season <2021]
train.head()

,season,round,country,url,driver,grid,podium,driver_points,driver_wins,driver_standings_pos,...,constructor_haas,constructor_lotus_f1,constructor_mclaren,constructor_mercedes,constructor_red_bull,constructor_renault,constructor_sauber,constructor_toro_rosso,constructor_toyota,constructor_williams
0,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,michael_schumacher,1,0,5.0,0.0,4.0,...,False,False,False,False,False,False,False,False,False,False
1,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,barrichello,2,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,montoya,3,0,8.0,0.0,2.0,...,False,False,False,False,False,False,False,False,False,True
3,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,frentzen,4,0,3.0,0.0,6.0,...,False,False,False,False,False,False,True,False,False,False
4,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,panis,5,0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False


In [1773]:
X_train = train.drop(['driver', 'country', 'podium', 'url'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: podium, dtype: int64

In [1774]:
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [1775]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential (
        nn.Flatten(),
        nn.Linear(69, 75),
        nn.ReLU(),
        nn.Linear(75, 25),
        nn.ReLU(),
        nn.Linear(25,50),
         nn.ReLU(),
        nn.Linear(50,10)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

In [1776]:
X_train = torch.FloatTensor(X_train.to_numpy())
y_train = torch.LongTensor(y_train)

In [1777]:
torch.manual_seed(42)
model = MLP()
model

MLP(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=69, out_features=75, bias=True)
    (2): ReLU()
    (3): Linear(in_features=75, out_features=25, bias=True)
    (4): ReLU()
    (5): Linear(in_features=25, out_features=50, bias=True)
    (6): ReLU()
    (7): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [1778]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), 
                            lr=1e-4)

In [1779]:

# Train our model!
# Epochs? (one run thru all the training data in our network)
epochs = 1500
losses = []
for i in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_function(y_pred, y_train) # predicted values vs the y_train
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(f'Epoch: {i} and loss: {loss}')

Epoch: 0 and loss: 2.1787984371185303
Epoch: 10 and loss: 2.161515474319458
Epoch: 20 and loss: 2.1445322036743164
Epoch: 30 and loss: 2.127397298812866
Epoch: 40 and loss: 2.1096062660217285
Epoch: 50 and loss: 2.0906176567077637
Epoch: 60 and loss: 2.0698416233062744
Epoch: 70 and loss: 2.0466277599334717
Epoch: 80 and loss: 2.020237684249878
Epoch: 90 and loss: 1.9897366762161255
Epoch: 100 and loss: 1.9541476964950562
Epoch: 110 and loss: 1.9124106168746948
Epoch: 120 and loss: 1.8635305166244507
Epoch: 130 and loss: 1.8065464496612549
Epoch: 140 and loss: 1.7407170534133911
Epoch: 150 and loss: 1.6655067205429077
Epoch: 160 and loss: 1.580146074295044
Epoch: 170 and loss: 1.4841986894607544
Epoch: 180 and loss: 1.3779499530792236
Epoch: 190 and loss: 1.262932538986206
Epoch: 200 and loss: 1.141969919204712
Epoch: 210 and loss: 1.0193579196929932
Epoch: 220 and loss: 0.8997246623039246
Epoch: 230 and loss: 0.7873974442481995
Epoch: 240 and loss: 0.6858111023902893
Epoch: 250 and lo

In [1780]:
def scorecard(season):
    df = data.copy()
    df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)
    score = 0
    count = 0
    for circuit in df[df.season == season]['round'].unique():
        count +=1
        test = df[(df.season == season) & (df['round'] == circuit) & (df['grid'] == 1)]
    
        winner = data[(data.season == season) & (data['round'] == circuit) & (data['podium'] == 1)].grid
        try:
            winner = winner.to_numpy()[0]
        except:
            winner = None
        
        X_test = test.drop(['driver', 'country', 'podium', 'url'], axis = 1)
        y_test = test.podium
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        X_test = torch.Tensor(X_test.to_numpy())
        with torch.no_grad():
            model.eval()
            prediction = model(X_test)
            prob = F.softmax(prediction, dim=1)
            top_p, top_class = prob.topk(1, dim = 1)
            if prediction.argmax().item() == y_test.to_numpy()[0]:
                score+=1
                print(f'CORRECTLY predicted the pole to {"win" if prediction.argmax().item() == 1 else "lose"} with {top_p}% confidence')
            else:
                print(f'INCORRECTLY predicted the pole to {"win" if prediction.argmax().item() == 1 else "lose"} with {top_p}% confidence')

    print(f'{score} out of {count} races')

In [1781]:
def scorecard_pole(season):
    score = 0
    count = 0
    predicted = 0
    for circuit in df[df.season == season]['round'].unique():
        count +=1
        try:
            winner = data[(data.season == season) & (data['round'] == circuit) & (data['podium'] == 1)].grid.to_numpy()[0]
        except:
            continue
        if winner == 1:
            score+=1
    print(f'{score}')

In [1805]:
szn = 2022

In [1806]:
scorecard_ts(szn)

INCORRECTLY predicted. P2 should win in round 2 (actual winner was P4)
CORRECTLY predicted. P1 should win in round 3 (actual winner was P1)
INCORRECTLY predicted. P2 should win in round 4 (actual winner was P1)
INCORRECTLY predicted. P1 should win in round 5 (actual winner was P3)
CORRECTLY predicted. P2 should win in round 6 (actual winner was P2)
INCORRECTLY predicted. P4 should win in round 7 (actual winner was P3)
CORRECTLY predicted. P3 should win in round 8 (actual winner was P3)
CORRECTLY predicted. P1 should win in round 9 (actual winner was P1)
INCORRECTLY predicted. P2 should win in round 10 (actual winner was P1)
INCORRECTLY predicted. P1 should win in round 11 (actual winner was P2)
CORRECTLY predicted. P2 should win in round 12 (actual winner was P2)
CORRECTLY predicted. P10 should win in round 13 (actual winner was P10)
CORRECTLY predicted. P14 should win in round 14 (actual winner was P14)
CORRECTLY predicted. P1 should win in round 15 (actual winner was P1)
CORRECTLY pr

In [1807]:
scorecard(szn)

INCORRECTLY predicted the pole to lose with tensor([[0.6495]])% confidence
CORRECTLY predicted the pole to lose with tensor([[0.8775]])% confidence
CORRECTLY predicted the pole to win with tensor([[0.7979]])% confidence
INCORRECTLY predicted the pole to lose with tensor([[0.8358]])% confidence
INCORRECTLY predicted the pole to win with tensor([[0.7787]])% confidence
CORRECTLY predicted the pole to lose with tensor([[0.6297]])% confidence
CORRECTLY predicted the pole to lose with tensor([[0.7846]])% confidence
CORRECTLY predicted the pole to lose with tensor([[0.6936]])% confidence
CORRECTLY predicted the pole to win with tensor([[0.9459]])% confidence
INCORRECTLY predicted the pole to lose with tensor([[0.8671]])% confidence
INCORRECTLY predicted the pole to win with tensor([[0.9530]])% confidence
CORRECTLY predicted the pole to lose with tensor([[0.7439]])% confidence
CORRECTLY predicted the pole to lose with tensor([[0.9584]])% confidence
CORRECTLY predicted the pole to lose with ten

In [1808]:
scorecard_pole(szn)

10
